In [ ]:
from IPython.display import clear_output

!pip install pytube
!pip install openai
!pip install google-api-python-client


clear_output(wait=False)

In [2]:
import googleapiclient.discovery
import googleapiclient.errors
import openai
from pytube import YouTube
import requests
from PIL import Image
import os
import time

openai.api_key = "----- ENTER YOUR OPENAI API KEY HERE -----"

api_service_name = "youtube"
api_version = "v3"

DEVELOPER_KEY = "----- ENTER YOUR YouTube Data API v3 API KEY HERE -----" #The developer key could be found by enabling YouTube Data API v3 services in Google Cloud

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

In [3]:
def transcribe_audio(audio_file):

    """
    Function to transcribe an audio file

            Parameters:
                    audio_file (bytes): The audio file in byte like format
            Returns:
                    transcribed_text (string): The transcription of the audio file in text format
    """
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    transcribed_text = transcript['text']
    return transcribed_text

In [4]:
# def get_text_from_image(image_file):
#     #Convert image to text
#     # img = Image.open(image_file)
#     text = pytesseract.image_to_string(img)

#     return text

In [5]:
def get_thumbnail(video_link):
  """
    Function to get thumbnail of a YouTube video

            Parameters:
                    video_link (string): Hyperlink for a YouTube video.
            Returns:
                    image: The thumbnail image of the video.
  """
  utube = YouTube(video_link)
  thumbnail_url = utube.thumbnail_url
  thumbnail_response = requests.get(thumbnail_url)

  with open('i.png','wb') as f:
    f.write(thumbnail_response.content)

  image = Image.open('i.png')
  os.remove('i.png')

  return image

In [6]:
def get_details(video_link):

    """
    Function to return details about a YouTube video given its hyperlink

          Parameters:
                  video_link (string): Hyperlink for a YouTube video.
                                       Example- https://www.youtube.com/shorts/VIDEO_ID or https://www.youtube.com/watch?v=VIDEO_ID

          Returns:
                  video_id (string): Unique identifier for the video. Example- YDc_wyAjHME
                  video_title (string): Title of the YouTube video
                  video_description (string): Description of the video
                  channel_name (string): Name of the YouTube channel
                  video_num_views (string): Number of views on the video
                  video_num_likes (string): Number of likes on the video
                  video_num_comments (string): Number of comments on the video
                  channel_num_subscribers (string): Number of subsribers for the YouTube channel
                  transcript (string): The transcript of the video



    """

    if video_link.find("shorts"):
      video_id = video_link.split("/")[-1]
    else:
      video_id = video_link.split("=")[1]


    #---------------------------------------
    request = youtube.videos().list(
    part=["statistics","snippet"],
    id=video_id
    )
    response = request.execute()

    video_title = response['items'][0]['snippet']['title']
    video_description = response['items'][0]['snippet']['description']
    channel_id = response['items'][0]['snippet']['channelId']
    video_num_views = response['items'][0]['statistics']['viewCount']
    video_num_likes = response['items'][0]['statistics']['likeCount']
    video_num_comments = response['items'][0]['statistics']['commentCount']

    #---------------------------------------
    request2 = youtube.channels().list(
    part=["snippet","statistics"],
    id=channel_id
    )
    response2 = request2.execute()

    channel_name = response2['items'][0]['snippet']['title']
    chanel_description = response2['items'][0]['snippet']['description']
    channel_num_subscribers = response2['items'][0]['statistics']['subscriberCount']
    channel_num_videos = response2['items'][0]['statistics']['videoCount']

    #---------------------------------------
    request3 = youtube.commentThreads().list(
    part="snippet",
    videoId=video_id,
    maxResults=10
    )

    response3 = request3.execute()
    video_comments = [item['snippet']['topLevelComment']['snippet']['textOriginal'] for item in response3['items']]

    #---------------EXTRACTING TRANSCRIPT----------------
    # Initialize a YouTube object
    yt = YouTube(video_link)
    audio_stream = yt.streams.filter(only_audio=True, file_extension="webm").last()
    audio_stream.download(filename='audio_temp.wav')
    audio_file =  open('audio_temp.wav', "rb")

    time.sleep(10)
    transcript = transcribe_audio(audio_file)
    audio_file.close()
    os.remove('audio_temp.wav')


    return [video_id, video_title, video_description, channel_name, video_num_views, video_num_likes,
            video_num_comments, channel_num_subscribers, transcript]

In [7]:
url = 'https://www.youtube.com/shorts/YDc_wyAjHME'

In [8]:
video_details = get_details(url)

In [9]:
video_details

['YDc_wyAjHME',
 'Please Don’t invest in stocks ! #sharemarket',
 '',
 'pranjal kamra',
 '5633373',
 '331054',
 '1465',
 '5640000',
 'हाहा, पता है, मैं ही बोल रहा हूँ, स्टोक्स में इन्वेस्ट नहीं करना चाहिए मैं ने जिसने 4 साल में रट लगा रखी है, कि स्टोक्स में इन्वेस्ट करो इसको थोड़ा सबचते हैं, अब देखो कमा लिया, तो 800 रुपे ही कमा लिया, तो 800 रुपे के लिए खुद का इतना टाइम क्यों लगाना, उस केस में मूच्यल फंड बेटर है स्टोक्स कब बेटर है, अगर किसी को इसमें करियर बनाना है, तो जरूर, आपको स्टोक मार्केट खुद करना चाहिए, आप तभी सीखोगे या लाखो करोर रुपे मैनेज करने हैं, तो खुद सीख के अगर इंवेस्ट करो, 2-4 परसंट एक्स्ट्रा आजाएं, तो इस वर्थ है, वो जो टाइम आपने दिया']